In [1]:
import z3
from lib.ts import Ts
from lib.prove import prove_inductive, prove
from lib.state_machine import smart_contract_state_machine

In [2]:

statemachine = smart_contract_state_machine('Crowdsale')
state, stateOut = statemachine.add_state("state", z3.BitVecSort(2))
deposits, depositsOut = statemachine.add_state("deposits", z3.ArraySort(z3.BitVecSort(256), z3.BitVecSort(256)))
totalDeposits, totalDepositsOut = statemachine.add_state("totalDeposits", z3.BitVecSort(256))
raised, raisedOut = statemachine.add_state("raised", z3.BitVecSort(256))
aux_claimrefund, aux_claimrefundOut = statemachine.add_state("aux_claimrefund", z3.BoolSort())
aux_withdraw, aux_withdrawOut = statemachine.add_state("aux_withdraw", z3.BoolSort())

GOAL = 10000
CLOSETIME = 10000


OPEN = 0
SUCCESS = 1
REFUND = 2




In [3]:
p = z3.BitVec('p',256)
now = statemachine.nowOut

statemachine.set_init(z3.And(
    z3.ForAll(p, deposits[p]==0),
    raised == 0,
    totalDeposits == 0,
    state == OPEN,
    aux_claimrefund == False,
    aux_withdraw == False
))


sender = z3.BitVec('sender',256)
value = z3.BitVec('amount',256)

statemachine.add_tr(
    tr_name = "invest",
    parameters = (value, sender),
    guard = z3.And(# now <= CLOSETIME,
                   raised < GOAL,
                   ),
    transfer_func = z3.And(raisedOut == raised + value,
                           depositsOut == z3.Update(deposits,sender,deposits[sender]+value),
                           totalDepositsOut == totalDeposits+value,
                        )
)

statemachine.add_tr(
    tr_name = "close_success",
    parameters = None,
    guard = z3.And(raised >= GOAL,
                   ),
    transfer_func = z3.And(
                           stateOut == SUCCESS,
                        )
)

statemachine.add_tr(
    tr_name = "close_refund",
    parameters = None,
    guard = z3.And(now > CLOSETIME,
                   raised < GOAL,
                   ),
    transfer_func = z3.And(stateOut == REFUND,
                        )
)

p = z3.BitVec('p',256)
statemachine.add_tr(
    tr_name = "claimrefund",
    parameters = (p, ),
    guard = z3.And(state == REFUND,
                   raised < GOAL,
                   ),
    transfer_func = z3.And(depositsOut == z3.Update(deposits,p,0),
                           totalDepositsOut == totalDeposits - deposits[p],
                           aux_claimrefundOut == True,
                        )
)


statemachine.add_tr(
    tr_name = "withdraw",
    parameters = None,
    guard = z3.And(state == SUCCESS,
                   ),
    transfer_func = z3.And(totalDepositsOut == 0,
                           aux_withdrawOut == True,
                        )
)
print(statemachine.transfer_func)


And(And(raised' == raised + amount,
        deposits' ==
        Store(deposits, sender, deposits[sender] + amount),
        totalDeposits' == totalDeposits + amount),
    func' == "invest")
And(And(state' == 1), func' == "close_success")
And(And(state' == 2), func' == "close_refund")
And(And(deposits' == Store(deposits, p, 0),
        totalDeposits' == totalDeposits - deposits[p],
        aux_claimrefund' == True),
    func' == "claimrefund")
And(And(totalDeposits' == 0, aux_withdraw' == True),
    func' == "withdraw")
{'invest': And(raised' == raised + amount,
    deposits' ==
    Store(deposits, sender, deposits[sender] + amount),
    totalDeposits' == totalDeposits + amount,
    func' == "invest",
    state' == state,
    aux_claimrefund' == aux_claimrefund,
    aux_withdraw' == aux_withdraw), 'close_success': And(state' == 1,
    func' == "close_success",
    deposits' == deposits,
    totalDeposits' == totalDeposits,
    raised' == raised,
    aux_claimrefund' == aux_claimrefund,

In [4]:
# print(statemachine.now_state)
# statemachine.transfer("invest", amount == 100)
# print(statemachine.now_state)

positive_traces = []
positive_traces.append(
    [
        ('invest', now == 0, sender == 0x114514, value == 100),
        ('invest', now == 0, sender == 0x114515, value == 100),
        ('invest', now == 0, sender == 0x114516, value == 100),
        ('invest', now == 0, sender == 0x114517, value == 100),
        ('invest', now == 0, sender == 0x114518, value == 100),
        ('invest', now == 0, sender == 0x114519, value == 100),
        ('invest', now == 0, sender == 0x114520, value == 100),
    ]
)

positive_traces.append(
    [
        ('invest', now == 0, sender == 0x114514, value == 100),
        ('invest', now == 1, sender == 0x114515, value == GOAL),
        ('close_success', now == 2),
        ('withdraw', now == 3)
    ]
)

positive_traces.append(
    [
        ('invest', now == 0, sender == 0x114514, value == 100),
        ('invest', now == 1, sender == 0x114515, value == GOAL),
        ('close_success', now == CLOSETIME + 1),
        ('withdraw', now == CLOSETIME + 2)
    ]
)

positive_traces.append(
    [
        ('invest', now == 0, sender == 0x114514, value == 100),
        ('invest', now == 1, sender == 0x114515, value == 100),
        ('close_refund', now == CLOSETIME + 1),
        ('claimrefund', now == CLOSETIME + 2, p == 0x114514)
    ]
)

negative_traces = []
negative_traces.append(
    [
        ('close_refund', now == CLOSETIME + 1),
        ('claimrefund', p == 0x114514, now == CLOSETIME + 2),
        ('invest', value == GOAL+1, now == CLOSETIME + 3),
        ('close_success', now == CLOSETIME + 4),
        ('withdraw', now == CLOSETIME + 5),
    ]
)

test_ntrace = negative_traces[0]
test_ptrace = positive_traces[0]
print(statemachine.simulate(test_ntrace, show_log=False))
r2 = z3.Not(z3.And(aux_withdraw, aux_claimrefund))
statemachine.bmc(z3.Not(r2))
statemachine.add_guard('invest', now <= CLOSETIME)
statemachine.bmc(z3.Not(r2))
# statemachine.simulate(trace, show_log=True)
print(statemachine.simulate(test_ntrace, show_log=False))



accept
[amount, sender, p]
iteration  0
iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
iteration  10
iteration  11
iteration  12
iteration  13
iteration  14
iteration  15
iteration  16
iteration  17
iteration  18
iteration  19
iteration  20
No model found!
reject


In [5]:
statemachine.clear_guards()
print(statemachine.condition_guards)

possible_guards = [
    state == OPEN, 
    state == SUCCESS, 
    state == REFUND, 
    now > CLOSETIME, 
    now < CLOSETIME, 
    raised >= GOAL, 
    raised < GOAL, 
]

result_guard = []

for tr in statemachine.transitions:
    for g in possible_guards:
        statemachine.add_guard(tr, g)
        # print(statemachine.condition_guards)
        if statemachine.simulate(test_ntrace, show_log=False) == 'reject':
            all_accept = True
            for ptrace in positive_traces:
                if statemachine.simulate(ptrace, show_log=False) == 'reject':
                    all_accept = False
                    break
            if all_accept:
                result_guard.append([tr, g])
        statemachine.clear_guards()

print(result_guard)


{'invest': True, 'close_success': True, 'close_refund': True, 'claimrefund': True, 'withdraw': True}
[['invest', state == 0], ['invest', now' < 10000], ['close_success', state == 0]]
